In [ ]:
#import Pkg; Pkg.add("JuMP")
#import Pkg; Pkg.add("HiGHS")
#import Pkg; Pkg.add("XLSX")
#import Pkg; Pkg.add("DataFrames")

In [1]:
# Used libraries
using JuMP, HiGHS, XLSX

In [2]:
# File to read
FileName = "Set_Or35_De70_Inst_1.xlsx" ;

In [3]:
# Basic data loading
 J = XLSX.readdata(FileName, "Parameters", "C1")  # Products (items)
 T = XLSX.readdata(FileName, "Parameters", "C2")  # Planning horizon
 M = XLSX.readdata(FileName, "Parameters", "C3")  # Machines
 K = XLSX.readdata(FileName, "Parameters", "C4")  # Orders

# Data reading
p  = Array{Float64, 2}(undef, J, 2)   # productivities
cp = Array{Float64, 2}(undef, J, 2)  # production costs
st = Array{Float64, 2}(undef, J, 2)  # setup times

PRO = XLSX.readdata(FileName, "Production", "A2:A$(J+1)")  # product
OPS = XLSX.readdata(FileName, "Production", "B2:B$(J+1)")  # manufacturing options
MA1 = XLSX.readdata(FileName, "Production", "C2:C$(J+1)")  # first choice for manufacturing
p[:,1]  = XLSX.readdata(FileName, "Production", "D2:D$(J+1)")  # productivity
cp[:,1] = XLSX.readdata(FileName, "Production", "E2:E$(J+1)")  # production cost
st[:,1] = XLSX.readdata(FileName, "Production", "F2:F$(J+1)")  # setup time
MA2 = XLSX.readdata(FileName, "Production", "G2:G$(J+1)")  # second choice for manufacturing 
p[:,2]  = XLSX.readdata(FileName, "Production", "H2:H$(J+1)")  # productivity
cp[:,2] = XLSX.readdata(FileName, "Production", "I2:I$(J+1)")  # production cost
st[:,2] = XLSX.readdata(FileName, "Production", "J2:J$(J+1)")  # setup time
ch = XLSX.readdata(FileName, "Production", "K2:K$(J+1)")   # inventory cost
s  = XLSX.readdata(FileName, "Production", "L2:L$(J+1)")   # initial stock 

MAQ = XLSX.readdata(FileName, "Machine", "A2:A$(M+1)")  # machine
n   = XLSX.readdata(FileName, "Machine", "B2:B$(M+1)")  # daily productive time
g   = XLSX.readdata(FileName, "Machine", "C2:C$(M+1)")  # maximum daily overtime
co  = XLSX.readdata(FileName, "Machine", "D2:D$(M+1)")  # overtime cost
ce  = XLSX.readdata(FileName, "Machine", "E2:E$(M+1)")  # extra shift cost
r  = XLSX.readdata(FileName, "Machine", "F2:F$(M+1)")   # maximum daily hours per extra shift

a   = XLSX.readdata(FileName, "Day", "B2:B$(T+1)")   # day, availability to work
u   = XLSX.readdata(FileName, "Day", "C2:C$(T+1)") ; # crew availability

# Data reading (orders)
xf = XLSX.readxlsx(FileName) 
sh = xf["Demand"]
ORD    = sh["A2:A$(K+1)"]    # order number
ORD_CL = sh["B2:B$(K+1)"]    # order customer
d      = sh["C2:C$(K+1)"]    # due date order
f      = sh["D2:D$(K+1)"]    # number of periods allowed to cover backorder
q      = sh["E2:V$(K+1)"]    # quantity of product j in order i, q[k,j]
cb     = sh["W2:AN$(K+1)"] ; # backorder cost (customer), cb[k,j]

In [4]:
# Data generation
aux = zeros(Int8, (M, J))   # indica en po(i,j) 0 no es opcion 1 primera 2 segunda
for j = 1:J
    for i = 1:M
        if cmp(MAQ[i],MA1[j])==0 
            aux[i,j]=1
        end
         if cmp(MAQ[i],MA2[j])==0
            aux[i,j]=2
        end
    end
end
 
G=1+16*max( maximum(p) ) ;  # big-M parameter

In [5]:
# Lot sizing model
LSModel = Model(HiGHS.Optimizer) ;

In [6]:
# Model variables
@variable(LSModel, X[1:J, 1:2, 1:T] >= 0) # quantity of product j manufactured in the choice-machine [1,2] on day t
@variable(LSModel, Y[1:J, 1:T, 1:K] >= 0) # quantity of product j, manufactured in period t, used to partially cover order k
@variable(LSModel, E[1:M, 1:T] >= 0)      # overtime used on machine i on day t
@variable(LSModel, Z[1:M, 1:T], Bin)      # indicates with value=1, if on day t an extra shift is used on machine I
@variable(LSModel, U[1:J, 1:2, 1:T], Bin) # indicates with value=1, if on day t product j is manufactured on machine I
@variable(LSModel, S[1:J, 1:K] >= 0)     # quantity of product j from the initial stock used to partially cover order k
@variable(LSModel, PCost >= 0)     # ...
@variable(LSModel, ICost >= 0)     # ...
@variable(LSModel, BCost >= 0)     # ...
@variable(LSModel, ECost >= 0)     # ...
@variable(LSModel, ZCost >= 0)   ;  # ...

In [7]:
# Objective Function
@constraint(LSModel, P_cost, sum( cp[j,1]*X[j,1,t]  for j in 1:J, t in 1:T ) 
        + sum( cp[j,2]*X[j,2,t]  for j in 1:J, t in 1:T  if OPS[j]==2 )  == PCost) 

@constraint(LSModel, I_cost, sum(Y[j,t,k]*ch[j]*(d[k]-t)  for j in 1:J, k in 1:K, t in 1:d[k]-1 ) 
        + sum(S[j,k]*ch[j]*(d[k]-1)  for j in 1:J, k in 1:K ) == ICost )

@constraint(LSModel, B_cost, sum(Y[j,t,k]*cb[k,j]*(t-d[k]) for j in 1:J, k in 1:K, t in d[k]+1:min(d[k]+f[k],T) ) == BCost )

@constraint(LSModel, E_cost, sum(E[i,t]*co[i]  for i in 1:M, t in 1:T ) == ECost )

@constraint(LSModel, Z_cost, sum(Z[i,t]*ce[i]  for i in 1:M, t in 1:T ) == ZCost  )

@objective(LSModel, Min, PCost + ICost + BCost + ECost + ZCost ) ;

In [8]:
# Constraints

In [9]:
# Restricts the manufacturing of products that only have one manufacturing option
@constraint(LSModel,prod_null, 
    sum(X[j,2,t] for j in 1:J, t in 1:T if OPS[j]==1) == 0 ) 

@constraint(LSModel,bin_prod_null, 
    sum(U[j,2,t] for j in 1:J, t in 1:T if OPS[j]==1) == 0 ) ;

In [10]:
# Production capacity (hours)
@expression(LSModel, setup[i = 1:M, t = 1:T],
           sum( st[j,aux[i,j]]*U[j,aux[i,j],t] for j in 1:J if aux[i,j]!=0) ) 

@expression(LSModel, capacity[i = 1:M, t = 1:T],
         a[t]*(n[i] + E[i,t] + Z[i,t]*r[i]) - setup[i,t] ) 

@constraint(LSModel, capacity_used[i = 1:M, t = 1:T], 
         sum(X[j,1,t]/p[j,1] for j =1:J if cmp(MAQ[i],MA1[j])==0) + sum(X[j,2,t]/p[j,2] for j =1:J if cmp(MAQ[i],MA2[j])==0)  <=  capacity[i,t]) ;

In [11]:
# Max. daily productive overtime (hours)
@constraint(LSModel, overtime_used[i = 1:M, t = 1:T],
        E[i,t] <= (1-Z[i,t])*g[i]   ) ;

In [12]:
# Max. daily hours per extra shift 
@constraint(LSModel, extra_shift_used[t = 1:T],
        sum(Z[i,t] for i in 1:M ) <= a[t]*u[t]   )  ;

In [13]:
# Consumption of the initial stock
@constraint(LSModel, initial_stock_used[j = 1:J],
        sum(S[j,k] for k in 1:K)   == s[j]  ) ;

In [14]:
# Production to order
@constraint(LSModel, production_used[j = 1:J, t = 1:T],
        X[j,1,t] + X[j,2,t]*(OPS[j]-1) == sum(Y[j,t,k] for k in 1:K if t<=d[k]+f[k])   ) ;

In [15]:
# Order fulfillment
@constraint(LSModel, order_fulfillment[j = 1:J, k = 1:K],
        S[j,k] + sum(Y[j,t,k] for t in 1:min(d[k]+f[k],T) ) == q[k,j] ) ;    

In [16]:
# Binary production for setups
@constraint(LSModel,bin_prod_ub[j = 1:J, t = 1:T, o = 1:2], 
    U[j,o,t] <= G*X[j,o,t] )

@constraint(LSModel,bin_prod_lb[j = 1:J, t = 1:T, o = 1:2], 
    G*U[j,o,t] >= X[j,o,t] ) ;

In [17]:
set_silent(LSModel)
optimize!(LSModel) ;

In [18]:
# OutputFile, print reports

OutName = string("Report_",FileName) 
if termination_status(LSModel) == OPTIMAL
    XLSX.openxlsx(OutName, mode="w") do xf
        sheet = xf[1]
        XLSX.rename!(sheet, "products_R")
        sheet[1,1] = "Day"
        sheet[1,2] = "Product"
        sheet[1,3] = "Machine first option"
        sheet[1,4] = "Quantity(1)"
        sheet[1,5] = "Machine second option"
        sheet[1,6] = "Quantity(2)"
        row=2
        for t = 1:T
            for j = 1:J
                sheet[row,1] = t
                sheet[row,2] = PRO[j]
                sheet[row,3] = MA1[j]
                sheet[row,4] = value(X[j,1,t])
                if OPS[j] == 2
                    sheet[row,5] = MA2[j]
                    sheet[row,6] = value(X[j,2,t])
                end
                row += 1 
            end  
        end        
           
        XLSX.addsheet!(xf, "machines_R")
        xf[2][1,1] = "Day"
        xf[2][1,2] = "Machine"
        xf[2][1,3] = "Overtime used"
        xf[2][1,4] = "Extra shift used"
        xf[2][1,5] = "Slack"
        xf[2][1,6] = "Production (item, quantity ...)"
        row=2
        for t = 1:T
            for i = 1:M
                xf[2][row,1] = t
                xf[2][row,2] = MAQ[i]
                xf[2][row,3] = value(E[i,t])
                if value(Z[i,t]) > 0.5
                    xf[2][row,4] = "YES"
                    else xf[2][row,4] = "no"
                end
                xf[2][row,5] = round(normalized_rhs(capacity_used[i,t]) - value(capacity_used[i,t]), digits = 6)
                col=6
                for j = 1:J
                    if aux[i,j] != 0 && value(X[j,aux[i,j],t]) != 0 
                        xf[2][row,col] = PRO[j]
                        xf[2][row,col+1] = value(X[j,aux[i,j],t])
                        col = col + 2
                    end
                end
                row += 1 
            end  
        end        
    
        XLSX.addsheet!(xf, "order_fulfillment_R")
        xf[3][1,1] = "Order"
        xf[3][1,2] = "Customer"
        xf[3][1,3] = "Due date"
        xf[3][1,4] = "Product"
        xf[3][1,5] = "Ordered_quantity"
        xf[3][1,6] = "Fulfillment (day/quantity): (day 0, represent initial stock)"
        row = 2
        for k = 1:K
            for j = 1:J
                xf[3][row,1] = ORD[k]
                xf[3][row,2] = ORD_CL[k]
                xf[3][row,3] = d[k]
                xf[3][row,4] = PRO[j]
                xf[3][row,5] = q[k,j]
                col = 6
                if q[k,j] != 0 
                    if value(S[j,k]) != 0
                        xf[3][row,col] = 0
                        xf[3][row,col+1] = value(S[j,k])
                        col = col + 2
                    end
                    for t = 1:min(d[k]+f[k],T)
                        if value(Y[j,t,k]) != 0
                            xf[3][row,col] = t
                            xf[3][row,col+1] = value(Y[j,t,k])
                            col = col + 2
                        end
                    end
                end
                row += 1
            end
        end
    
        XLSX.addsheet!(xf, "costs_R")
        xf[4][1,1] =  "Production cost"
        xf[4][2,1] =  "Inventory cost"
        xf[4][3,1] =  "Backorder cost"
        xf[4][4,1] =  "Overtime cost"
        xf[4][5,1] =  "Extra-shift cost"
        xf[4][6,1] =  "Total cost"
        xf[4][1,2] =  value.(PCost)
        xf[4][2,2] =  value.(ICost)
        xf[4][3,2] =  value.(BCost)
        xf[4][4,2] =  value.(ECost)
        xf[4][5,2] =  value.(ZCost)
        xf[4][6,2] =  objective_value(LSModel)
    
        # Control sheet
        XLSX.addsheet!(xf, "control_R")
        for j = 1:J
            xf[5][1+j,1] =  "Backorder (ton)"
            xf[5][1+j,2] =  PRO[j]
        end
        for t = 1:T
            sref = string("day ",t)
            xf[5][1,t+2] =  sref
        end
        back = zeros(Float16, (J,T)) 
        for j = 1:J, t = 1:T
            for k = 1:K
                if t == d[k]
                    aux_value = 0.0
                    init = 1 + d[k]
                    fini = min(d[k]+f[k],T)
                    if init <= fini
                        aux_value = sum(value(Y[j,tt,k]) for tt in init:fini )
                    end
                    back[j,t] += aux_value
                end
            end
            xf[5][1+j,t+2] = back[j,t]
        end
    
        for i = 1:M
            xf[5][1+J+i,1] =  "Slack (hrs)"
            xf[5][1+J+i,2] =  MAQ[i]
        end
        for i = 1:M, t = 1:T
            xf[5][1+J+i,t+2] = round(normalized_rhs(capacity_used[i,t]) - value(capacity_used[i,t]), digits = 6)
        end
    
        XLSX.addsheet!(xf, "Sol_summ_R")
        xf[6][1,1] = string(solution_summary(LSModel))
        xf[6][3,1] = "termination_status"
        xf[6][3,2] = string(termination_status(LSModel))
        xf[6][4,1] = "objective_value"
        xf[6][4,2] = objective_value(LSModel)
        xf[6][5,1] = "result_count"
        xf[6][5,2] = result_count(LSModel)
        xf[6][6,1] = "solve_time (sec.)"
        xf[6][6,2] = solve_time(LSModel)

        

        XLSX.addsheet!(xf, "Stock_R")    
        for j = 1:J
            xf[7][j,1] = PRO[j]
            stock_level = 0
            for t = 1:T
                if t==1  stock_level = s[j]        + value(X[j,1,1]) + value(X[j,2,1])
                else     stock_level = stock_level + value(X[j,1,t]) + value(X[j,2,t])
                end
                for k = 1:K
                    if d[k]==t   stock_level = stock_level - q[k,j]
                    end
                end
                xf[7][j,t+1] = stock_level
            end
        end

                
    end # end do
    
else print(FileName," Not solved")
end ;



In [ ]:
set_attribute(LSModel, "time_limit", 60.0)

In [ ]:
optimize!(LSModel) 